In [20]:
import xlrd
import itertools

In [21]:
phase_lengths = {'early G1': 27.2, 'late G1': 33.7, 'S': 20.5, 'G2': 21.4, 'P/M': 9.8, 'Ana': 10.4, 'T/C': 4.1}
sum(phase_lengths.values())

127.1

Method to split a list at a certain value (http://stackoverflow.com/questions/4322705/split-a-list-into-nested-lists-on-a-value):

In [26]:
def isplit(iterable,splitters):
    return [list(g) for k, g in itertools.groupby(iterable, lambda x:x in splitters) if not k]

In [32]:
mrna_file = '../data/Sup_PivotTable_RNA_forPubli_AllGenes.xlsx'
workbook = xlrd.open_workbook(mrna_file)

sheetnames = ['SIC1', 'CLN2', 'CLB5']
for sheet in sheetnames:
    worksheet = workbook.sheet_by_name(sheet)
    num_rows = worksheet.nrows - 1
    num_cells = worksheet.ncols
    print sheet
    print isplit(worksheet.col_values(0), ('',))[0][1: -1] 
    for col in range(1, 9):
        print isplit(worksheet.col_values(col), ('',))[0][:-1]

SIC1
[u'# RNA/cell', 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 22.0, 23.0, 24.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 34.0, 35.0, 36.0, 38.0, 39.0, 40.0, 51.0, 56.0, 66.0]
[u' # of cells', 102.0, 200.0, 245.0, 139.0, 58.0, 47.0, 23.0, 20.0, 13.0, 10.0, 19.0, 8.0, 9.0, 7.0, 6.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 2.0, 2.0, 3.0, 3.0, 2.0, 3.0, 5.0, 5.0, 2.0, 6.0, 2.0, 3.0, 2.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]
[u'early G1', 22.0, 35.0, 48.0, 30.0, 10.0, 11.0, 3.0, 5.0, 7.0, 4.0, 8.0, 4.0, 5.0, 1.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[u'late G1', 24.0, 53.0, 74.0, 52.0, 19.0, 14.0, 10.0, 6.0, 2.0, 3.0, 6.0, 2.0, 2.0, 1.0, 1.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
[u'S', 22.0, 41.0, 54.0, 22.0, 9.0, 12.0, 2.0, 2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 